In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
images = tfds.load(name="colorectal_histology", split="train", as_supervised=True)
images = images.batch(16)
images = images.map(lambda img,lab: (tf.image.convert_image_dtype(img, dtype=tf.float32),lab))
images = images.map(lambda img,lab: (img,img))

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/colorectal_histology/2.0.0.incompletePVYUSE/colorectal_histology-train.tfrecord


  0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset colorectal_histology downloaded and prepared to /root/tensorflow_datasets/colorectal_histology/2.0.0. Subsequent calls will reuse this data.


In [ ]:
print(type(images))

<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>


In [ ]:
import keras
from keras import layers

input_img = keras.Input(shape=(150, 150, 3))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (19, 19, 8) i.e. 128-dimensional

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.ZeroPadding2D(padding=(1, 1))(x)
decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [ ]:
autoencoder.summary()

In [ ]:
autoencoder.fit(images,
                epochs=150,
                batch_size=128,
                shuffle=True,)

In [ ]:
# decoded_imgs = autoencoder.predict(images)
n = decoded_img.shape[0]
plt.figure(figsize=(100, 20))
i = 1

for img_b, label_b in images.take(1):     
    decoded_img = autoencoder.predict(img_b)
    
    for i in range(decoded_img.shape[0]):
        j = i + 1
        
        # Display original
        ax = plt.subplot(2, n, j)
        plt.imshow(img_b[i,:,:,:])
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # Display reconstruction
        ax = plt.subplot(2, n, j + n)
        plt.imshow(decoded_img[i,:,:,:])
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
plt.show()